<a href="https://colab.research.google.com/github/DiNOV-Tokyo/yolov5/blob/main/DL_torchvsion_ResNet50_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.ResNet50でファインチューニング・推論

参考にしたサイト# https://gurutaka-log.com/pytorch-lightning-fine-tuning



## 1.1 Google Drive をマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks
#%mkdir yolo_train
%cd yolo_train

/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/yolo_train


In [ ]:
# pytorch_lightningのインストール
!pip install pytorch_lightning 

     |████████████████████████████████| 915 kB 8.4 MB/s 
     |████████████████████████████████| 234 kB 28.8 MB/s 
     |████████████████████████████████| 118 kB 29.0 MB/s 
     |████████████████████████████████| 10.6 MB 26.0 MB/s 
     |████████████████████████████████| 829 kB 46.2 MB/s 
     |████████████████████████████████| 636 kB 50.0 MB/s 
     |████████████████████████████████| 1.3 MB 23.8 MB/s 
     |████████████████████████████████| 294 kB 61.1 MB/s 
     |████████████████████████████████| 142 kB 74.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=e6fdfd4fc2b757a65d1bd93a8d22f9dbfa3a3dc0dabadf0906d3849e6882e037
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstal

## 1.2 モジュール読み込み

In [ ]:
# pytorch関係
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer

# フォルダ内のデータ読込で使用
from PIL import Image
import glob


## 1.3 データパス設定

In [ ]:
# 画像読み取り
fold_path = 'dataset/train/'
imgs = []
for imgs_path in glob.glob(fold_path + '*'):
    imgs.append(glob.glob(imgs_path + '/*'))


In [ ]:
print(imgs)

[['dataset/train/gym/P1130514.JPG', 'dataset/train/gym/P1130515.JPG', 'dataset/train/gym/P1130516.JPG', 'dataset/train/gym/P1130517.JPG', 'dataset/train/gym/P1130518.JPG', 'dataset/train/gym/P1130519.JPG', 'dataset/train/gym/P1130520.JPG', 'dataset/train/gym/P1130521.JPG', 'dataset/train/gym/P1130522.JPG', 'dataset/train/gym/P1130523.JPG', 'dataset/train/gym/P1130524.JPG', 'dataset/train/gym/P1130525.JPG', 'dataset/train/gym/P1130526.JPG', 'dataset/train/gym/P1130527.JPG', 'dataset/train/gym/P1130528.JPG', 'dataset/train/gym/P1130529.JPG', 'dataset/train/gym/P1130530.JPG', 'dataset/train/gym/P1130531.JPG', 'dataset/train/gym/P1130532.JPG', 'dataset/train/gym/P1130533.JPG', 'dataset/train/gym/P1130534.JPG', 'dataset/train/gym/P1130535.JPG', 'dataset/train/gym/P1130536.JPG', 'dataset/train/gym/P1130537.JPG', 'dataset/train/gym/P1130538.JPG', 'dataset/train/gym/P1130539.JPG', 'dataset/train/gym/P1130540.JPG', 'dataset/train/gym/P1130541.JPG', 'dataset/train/gym/P1130542.JPG', 'dataset/tra

## 1.4 モデル作成　ResNet50

In [ ]:
# 学習済みモデル「ResNet50」をインポート
from torchvision.models import resnet50

# 学習済みモデル「ResNet50」をダウンロード
resnet = resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


## 1.5 transform作成

In [ ]:
#transform = transforms.Compose(
#    [
#        transforms.Resize(256),  # (256, 256) で切り抜く。
#        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
 #       transforms.ToTensor(),  # テンソルにする。
 #       transforms.Normalize(
 #           mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
 #       ),  # 標準化する。
 #   ]
#)


transform = transforms.Compose([
    # 224×224にリサイズ
    transforms.Resize((224, 224)),
    # torch.Tensor型に変換
    transforms.ToTensor(),
    # 学習済みモデルで使われる平均と標準偏差で標準化
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
])

## 1.6 torch.Tensor型に変更

In [ ]:
labels = []
img_datas = torch.tensor([])

# 画像データを配列に格納
# torch型に変更
for i,imgs_arr in enumerate(imgs):
    
    for img_path in imgs_arr:
        # ラベルを追加
        labels.append(i)

        # imgデータ（PIL型）の取得
        img = Image.open(img_path)

        # imgデータの前処理
        tensor_img = transform(img)

        # バッチサイズを追加: バッチサイズ, チャンネル, 高さ、幅
        tensor_img = tensor_img.unsqueeze(0)

        # torch.tensorのappend
        img_datas = torch.cat([img_datas, tensor_img],dim=0)

 # ラベルをTensorに変更
labels = torch.tensor(labels)

train val に分けて、Dataset準備

In [ ]:
# データセット化
datasets = torch.utils.data.TensorDataset(img_datas, labels)

# 訓練:検証 = 95% : 5%
n_train = int(len(datasets) * 0.95)
n_val = len(datasets) - n_train

# 擬似乱数のシード固定
torch.manual_seed(0)

# データセットを訓練用、検証用で分割
train,val = torch.utils.data.random_split(datasets,[n_train,n_val])

In [ ]:
type(img_datas)

torch.Tensor

## 1.7 モデル学習

In [ ]:
# 訓練用のクラス
class TrainNet(pl.LightningModule):

#    @pl.data_loader
    def train_dataloader(self):
        # ミニバッチ
        return torch.utils.data.DataLoader(train, self.batch_size,shuffle=True)
    
    def training_step(self, batch, batch_nb):
        # 入力と目標値を分割
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

# 検証用のクラス
class ValidationNet(pl.LightningModule):

#    @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        results =  {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

# 学習用の全体のクラス
# 訓練用、検証用を継承
class FineTuningNet(TrainNet, ValidationNet):

    # データ数が少ないのでbatch_sizeは大きめ
    def __init__(self, batch_size=256):
        super().__init__()
        self.batch_size = batch_size
        # 畳み込みの部分は学習済みモデルを活用
        self.conv = resnet50(pretrained=True)
        # 全結合層は1000 => 100 => 3
        # 1000はresnet18の仕様
        # 3は分類数（かしゆか、のっち、あーちゃん）
        self.fc1 = nn.Linear(1000, 100)
        self.fc2 = nn.Linear(100, 3)

        # 学習済みのパラメータを固定
        for param in self.conv.parameters():
            param.requires_grad = False

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# 再現性の確保
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 学習
fine_net = FineTuningNet()
trainer = Trainer(gpus=1, max_epochs=50)
trainer.fit(fine_net)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | conv | ResNet | 25.6 M
1 | fc1  | Linear | 100 K 
2 | fc2  | Linear | 303   
--------------------------------
100 K     Trainable params
25.6 M    Non-trainable params
25.7 M    Total params
102.630   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


メトリックの確認

In [ ]:
trainer.callback_metrics
#trainer.checkpoint_callback

{}

## 1.8 推論

In [ ]:
# unsqueeze：バッチサイズ1を追加
# cuda:GPUなので必要
x = val[0][0].unsqueeze(0).cuda()

# モデルモードを学習→推論に変更
fine_net.eval()
fine_net.freeze()

F.softmax(fine_net(x)), val[0][1]

RuntimeError: ignored

## 1.9 モデル保存

In [ ]:
# 学習済みモデルの保存
torch.save(fine_net.state_dict(), 'fine.pt')

ロードするとき

In [42]:
# モデルの定義
fien_net_pred = FineTuningNet()

# 推論モード
fien_net_pred.eval()
fien_net_pred.freeze()

# 重みの読み込み
fien_net_pred.load_state_dict(torch.load('fine.pt'))

<All keys matched successfully>

In [47]:
# サイズ確認
x = train[0][0].unsqueeze(0) 
y_predict = fien_net_pred(x)
F.softmax(y_predict), train[1][1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


(tensor([[0.9921, 0.0052, 0.0026]]), tensor(1))